In [611]:
# import Q lookup 式兩份問卷人工對應的表
import pandas as pd
Q_lookup = pd.read_excel("../../Q_lookup.xlsx")

In [612]:
# take a look
Q_lookup.head()

,rank,99-98,100-101
0,0,NaN,時間戳記
1,0,請問您的學號？,請問您的學號？(B101100100)
2,0,Network ID,NaN
3,1,本系之教育使我有機會參與醫學科學研究,1 整體醫學教育 [本系之教育使我有機會參與醫學科學研究]
4,1,本系之教育使我學到具醫療專業素養之醫療照護,1 整體醫學教育 [本系之教育使我學到具醫療專業素養之醫療照護]


In [613]:
# import questionaire original file
tmu_9899 = pd.read_excel('../../TMU_9998.xlsx')
tmu_102 = pd.read_excel('../../GQ_102101.xlsx') # diff question

# import other school
fju_9899 = pd.read_excel('../../FJU_9998.xlsx')
mmc_9899 = pd.read_excel('../../MMC_9998.xlsx')
kmu_9899 = pd.read_excel('../../KMU_9998.xlsx')

# add school tag
tmu_9899['school'] = '北醫'
tmu_102['school'] = '北醫'
fju_9899['school'] = '輔仁'
mmc_9899['school'] = '馬偕'
kmu_9899['school'] = '高醫'

In [614]:
# hosital tag for TMU

# import data from 系辦 (資料已經人工清理過：例如：附醫-->北醫；臺大醫院 --> 台大 以求三屆的標籤相同才能電腦比較)
hp101 = pd.read_excel('../../991_981_101_hospital.xlsx', sheet_name = '101')
hp99 = pd.read_excel('../../991_981_101_hospital.xlsx', sheet_name = '991')
hp98 = pd.read_excel('../../991_981_101_hospital.xlsx', sheet_name = '981') # all upper case

# 注意學號大小寫會造成 join 不起來 也移除空白和'
tmu_102['請問您的學號？(B101100100)'] = tmu_102['請問您的學號？(B101100100)'].str.upper().str.replace(' ','').str.replace("\'", '').str.replace("\n", '')
print('學號重複', tmu_102.duplicated(subset= '請問您的學號？(B101100100)').shape)
tmu_102.drop_duplicates(subset= '請問您的學號？(B101100100)', keep = 'last', inplace = True) # 有重複填寫的人
tmu_9899['請問您的學號？'] = tmu_9899['請問您的學號？'].str.upper().str.upper().str.replace(' ','').str.replace("\'", '').str.replace("\n", '')

# 醫院表裡的學號也都有空白影響alignment
for hp in [hp101, hp99, hp98]:
    hp['student_id'] = hp['student_id'].str.replace(' ','').str.replace("\'", '').str.replace("\n", '')
# tag year
hp101['year'] = 101
hp98['year'] = 98
hp99['year'] = 99

# tag hospital
new_102 = tmu_102.merge(hp101, left_on = '請問您的學號？(B101100100)', right_on = 'student_id', how = 'left')
new_99 = tmu_9899.merge(hp99, left_on ='請問您的學號？', right_on = 'student_id', how = 'inner')
new_98 = tmu_9899.merge(hp98, left_on ='請問您的學號？', right_on = 'student_id', how = 'inner')

學號重複 (377,)


In [615]:
all_9899 = pd.concat([new_99, new_98, fju_9899, mmc_9899, kmu_9899], ignore_index = True, sort = False)

In [616]:
all_9899.groupby(by = 'year')['實習醫院'].value_counts()

year  實習醫院
98.0  北萬雙     54
      奇美       6
      林長       6
      北榮       5
      國泰       5
      馬偕       5
      台大       4
      新光       3
      北慈       1
      成大       1
99.0  雙和      24
      萬芳      20
      北醫      15
      台大       9
      國泰       6
      奇美       6
      馬偕       6
      新光       5
      北榮       3
      高長       3
      北慈       2
      林長       2
Name: 實習醫院, dtype: int64

In [617]:
# 清理 102 資料

# for simplicity, import 102 contact to find real 102 因為用入學年份粗分六七年制會很髒(有轉系、有擋修臨床而延後等等)，最後使用102及通訊錄中的學號(包含轉系生)來選出六年制的人是那些
contact = pd.read_excel('../../102contacts.xlsx')
true_six_year = [x.upper() for x in contact['Student_id']] # lowercase 都改小寫

new_102['six_year'] = new_102['請問您的學號？(B101100100)'].isin(true_six_year) # six_year 這個變數儲存的是：這個學號是否出現在 102的通訊錄裡。若 six_year = True 這就是六年制的同學(嗨麻吉)
new_102.loc[new_102['請問您的學號？(B101100100)'].isin(true_six_year), 'year'] = 102
new_102.loc[new_102['請問您的學號？(B101100100)'].isin(true_six_year), '實習醫院'] = '北萬雙'

In [618]:
new_102['six_year'].sum()

139

In [619]:
# 保守估計 100 的人若學號有 B101100 則算成 100
no_year = new_102.loc[new_102['year'].isnull(), '請問您的學號？(B101100100)']
new_102.loc[no_year.str.contains('B101100').index, 'year'] = 100

In [620]:
# 另外是 new_102 這一個問卷的選項把它改成 12345

# change scaling to 1,2,3,4,5 (非常不滿意，不滿意，普通，滿意，非常不滿意)
# 此份問卷不同的問題有不同的中文表達滿意程度，在這裡我通通把它轉成 1-5 分 (這是非常惱人的設計問卷的方式!)
scaling = {'非常不滿意':1,
          '不滿意':2,
          '普通':3,
          '滿意':4,
          '非常滿意':5,
          '良好':4,
          '非常良好':5,
          '不佳':2,
          '非常不佳':1,
          '非常清楚':5,
          '非常不清楚':1,
          '清楚':4,
          '不清楚':2}
new_102.iloc[:, 2:-2] = new_102.iloc[:, 2:-2].applymap(lambda x: scaling[x] if x in scaling.keys() else x) # 若問題是用1-5芬可以轉換的就轉換，不能轉的就保留原本的文字(像是：我的第一志願是？精神科﹔則保留"精神科")

In [621]:
# 接下來進行最艱鉅的任務：將兩份問題不一樣的問卷對齊 :( 拜託以後用同一份問卷同一個平台收集好不好

# 首先，102 這份問卷其實在發出去之後有改過，造成大量 NaN
# 因此透過 groupby 對應好的 9899 來顯示出哪些其實是相同但是改過的題目
Q_lookup.loc[Q_lookup.duplicated(['99-98'], keep = False)].dropna()

,rank,99-98,100-101
21,5,本系小組討論課程的內容合宜,2-4: 小組討論課程（ PBL等小組討論課程 ） [內容合宜]
22,5,本系小組討論課程的份量適中,2-4: 小組討論課程（ PBL等小組討論課程 ） [份量適中]
23,5,本系小組討論課程有與臨床應用整合,2-4: 小組討論課程（ PBL等小組討論課程 ） [有與臨床應用整合]
24,5,本系小組討論課程對我很有幫助,2-4: 小組討論課程（ PBL等小組討論課程 ） [對我很有幫助]
25,5,本系小組討論課程的內容合宜,2-5: 小組討論課程（ PBL等小組討論課程 ） [內容合宜]
26,5,本系小組討論課程的份量適中,2-5: 小組討論課程（ PBL等小組討論課程 ） [份量適中]
27,5,本系小組討論課程有與臨床應用整合,2-5: 小組討論課程（ PBL等小組討論課程 ） [有與臨床應用整合]
28,5,本系小組討論課程對我很有幫助,2-5: 小組討論課程（ PBL等小組討論課程 ） [對我很有幫助]


In [622]:
new_102[['2-4: 小組討論課程（ PBL等小組討論課程 ） [份量適中]', '2-5: 小組討論課程（ PBL等小組討論課程 ） [份量適中]']].count()

2-4: 小組討論課程（ PBL等小組討論課程 ） [份量適中]    319
2-5: 小組討論課程（ PBL等小組討論課程 ） [份量適中]     36
dtype: int64

In [623]:
new_102[['2-4: 小組討論課程（ PBL等小組討論課程 ） [內容合宜]', '2-5: 小組討論課程（ PBL等小組討論課程 ） [內容合宜]']].count() #看起來是 2-5 重複了，所以後來被拿掉

2-4: 小組討論課程（ PBL等小組討論課程 ） [內容合宜]    319
2-5: 小組討論課程（ PBL等小組討論課程 ） [內容合宜]     36
dtype: int64

In [624]:
cols_to_remove = Q_lookup.iloc[25:29]

In [625]:
new_102.shape # 移除前

(319, 178)

In [626]:
new_102.drop(columns = cols_to_remove['100-101'], inplace = True)

In [627]:
new_102.shape # 移除後

(319, 174)

In [628]:
Q_lookup.shape # 移除前

(174, 3)

In [629]:
# 問卷對應的表格 Qlookup 也要丟掉
Q_lookup.drop(index = [25,26,27,28], inplace = True)

In [630]:
Q_lookup.shape # 移除後

(170, 3)

In [631]:
all_9899.shape # map 以前

(254, 126)

In [632]:
# 接下來將所有 9899 的問題都改成 102 的
new_lookup = Q_lookup.dropna()
new_lookup.set_index('99-98', inplace = True)

def mapper(x):
    if x in new_lookup.index:
        return(new_lookup.loc[x, '100-101'])
    else:
        return(x) # keep original value
mapped_9899 = all_9899.rename(columns = mapper)

In [633]:
mapped_9899.shape # map 以後

(254, 126)

In [634]:
# 將兩份表格接在一起
all_data = pd.concat([mapped_9899, new_102], sort = False)

In [635]:
all_data.shape

(573, 182)

In [636]:
all_data.loc[:, all_data.count() == 663].columns

Index([], dtype='object')

In [637]:
all_data.loc[:, all_data.count() < 663].columns

Index(['#', '請問您的學號？(B101100100)', '1 整體醫學教育 [清楚本系及各學科的教學目標]',
       '1 整體醫學教育 [本系之教學目標、教學方法、評分有互相配合]', '1 整體醫學教育 [本系之醫學知識、臨床技能與人文教育並重3 列]',
       '1 整體醫學教育 [本系之教育使我學到具醫療專業素養之醫療照護]', '1 整體醫學教育 [本系之教育使我有機會參與醫學科學研究]',
       '1 整體醫學教育 [我很滿意本系的醫學教育品質]', '2-1: 課程-通識課程 [必修課外，有足夠的時間可安排修通識課程]',
       '2-1: 課程-通識課程 [學分下限適中]',
       ...
       '3-5: 在臨床實習時，各科安排的夜間實習（含過夜實習）有按照臨床實習原則？ [泌尿科]',
       '3-5: 在臨床實習時，各科安排的夜間實習（含過夜實習）有按照臨床實習原則？ [骨科]',
       '3-5: 在臨床實習時，各科安排的夜間實習（含過夜實習）有按照臨床實習原則？ [麻醉科]',
       '3-5: 在臨床實習時，各科安排的夜間實習（含過夜實習）有按照臨床實習原則？ [放射科]',
       '4-1: 在學期間 [學生對於學業、生活與生涯輔導的滿意度]', '4-1: 在學期間 [學生對經濟援助諮詢和服務的滿意度]',
       '4-1: 在學期間 [醫學生認為他們有被不當對待經驗（沒有請選非常滿意）]', '4-1: 在學期間 [醫學系對於學校空間使用滿意度]',
       '其他專科醫師志願（請填寫：第幾志願/xxx）', 'six_year'],
      dtype='object', length=182)

In [638]:
all_data.to_excel('../../all_data.xlsx')